# Complete VWAP Trading Strategy Analysis
## Comprehensive Backtesting and Optimization of VWAP + Candlestick Pattern Strategies

This notebook implements a complete trading strategy analysis framework combining VWAP (Volume Weighted Average Price) with various candlestick patterns for systematic backtesting, optimization, and performance evaluation.

## 1. Environment Setup and Data Loading

In [ ]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import talib as ta
import random
import warnings
from datetime import datetime
import time
from typing import List, Dict, Tuple, Optional
import concurrent.futures
from pathlib import Path

# Backtesting libraries
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
# from backtesting.test import random_ohlc_data

import warnings
warnings.filterwarnings("ignore")

# Plotting libraries
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import seaborn as sns
from IPython.display import display, HTML

# Optimization and statistics
from scipy import stats
from sklearn.model_selection import TimeSeriesSplit

# Configure warnings and display settings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("✅ All libraries imported successfully")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"📈 TA-Lib available: {hasattr(ta, 'CDLHAMMER')}")

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Load and validate data
def load_market_data(file_path: str) -> pd.DataFrame:
    """
    Load market data from CSV with proper error handling
    """
    try:
        # Load the CSV with MultiIndex columns (Tickers, OHLCV)
        df = pd.read_csv(file_path, header=[0,1], index_col=0)
        
        # Drop any rows that are completely NaN
        df = df.dropna(how='all')
        
        # Convert all values to float
        df = df.astype(float)
        
        # Convert index to datetime
        df.index = pd.to_datetime(df.index)
        
        print(f"✅ Data loaded successfully: {df.shape}")
        print(f"📅 Date range: {df.index.min()} to {df.index.max()}")
        print(f"🏢 Tickers: {list(df.columns.levels[0])}")
        
        return df
        
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
        raise
    except Exception as e:
        print(f"❌ Error loading data: {str(e)}")
        raise

# Load the data
data_path = "../00_data/data_AAPL_MSFT_GOOGL_AMZN_TSLA_META_NVDA_2020-01-01_2025-10-26_raw.csv"
df_raw = load_market_data(data_path)

# Display data info
print("\n📋 Data Overview:")
df_raw.head()

❌ File not found: ../00_data/data_AAPL_MSFT_GOOGL_AMZN_TSLA_META_NVDA_2020-01-01_2025-10-26_raw.csv


FileNotFoundError: [Errno 2] No such file or directory: '../00_data/data_AAPL_MSFT_GOOGL_AMZN_TSLA_META_NVDA_2020-01-01_2025-10-26_raw.csv'

## 2. Data Preprocessing and Ticker Selection

In [ ]:
def select_random_ticker(df: pd.DataFrame, seed: Optional[int] = None) -> Tuple[str, pd.DataFrame]:
    """
    Select a random ticker and return cleaned data
    """
    if seed:
        random.seed(seed)
        np.random.seed(seed)
    
    # Get available tickers
    tickers = list(df.columns.levels[0])
    selected_ticker = random.choice(tickers)
    
    print(f"🎯 Randomly selected ticker: {selected_ticker}")
    
    # Extract data for selected ticker
    ticker_data = df.xs(selected_ticker, level=0, axis=1)
    
    # Clean column names (lowercase)
    ticker_data.columns = [col.lower() for col in ticker_data.columns]
    
    # Remove any remaining NaN values
    ticker_data = ticker_data.dropna()
    
    print(f"📊 Clean data shape: {ticker_data.shape}")
    print(f"📅 Date range: {ticker_data.index.min()} to {ticker_data.index.max()}")
    
    return selected_ticker, ticker_data

# Select ticker and prepare data
RANDOM_SEED = 42  # For reproducibility
ticker_name, df_clean = select_random_ticker(df_raw, seed=RANDOM_SEED)

# Display cleaned data
print(f"\n📈 Sample data for {ticker_name}:")
display(df_clean.head())
print(f"\n📊 Data statistics:")
display(df_clean.describe())

## 3. Indicator Calculation Functions

In [ ]:
# Define all indicator calculation functions
def calculate_vwap(df: pd.DataFrame) -> pd.Series:
    """Calculate VWAP (Volume Weighted Average Price)"""
    try:
        typical_price = (df['high'] + df['low'] + df['close']) / 3
        vwap = (df['volume'] * typical_price).cumsum() / df['volume'].cumsum()
        return vwap
    except Exception as e:
        print(f"❌ Error calculating VWAP: {e}")
        return pd.Series(index=df.index, dtype=float)

def calculate_atr(df: pd.DataFrame, period: int = 14) -> pd.Series:
    """Calculate ATR (Average True Range)"""
    try:
        high_low = df['high'] - df['low']
        high_close = (df['high'] - df['close'].shift()).abs()
        low_close = (df['low'] - df['close'].shift()).abs()
        true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        atr = true_range.rolling(window=period).mean()
        return atr
    except Exception as e:
        print(f"❌ Error calculating ATR: {e}")
        return pd.Series(index=df.index, dtype=float)

def calculate_candlestick_pattern(df: pd.DataFrame, pattern_name: str) -> pd.Series:
    """Calculate a specific candlestick pattern"""
    try:
        pattern_func = getattr(ta, pattern_name)
        result = pattern_func(df['open'], df['high'], df['low'], df['close'])
        return pd.Series(result, index=df.index)
    except Exception as e:
        print(f"❌ Error calculating {pattern_name}: {e}")
        return pd.Series(index=df.index, dtype=float)

# Define all candlestick patterns
CANDLESTICK_PATTERNS = [
    # Bullish reversal patterns
    'CDLHAMMER', 'CDLINVERTEDHAMMER', 'CDLMORNINGSTAR', 'CDLMORNINGDOJISTAR',
    'CDLENGULFING', 'CDLPIERCING', 'CDLHARAMI', 'CDLHARAMICROSS', 'CDLTAKURI',
    
    # Bullish continuation patterns  
    'CDL3WHITESOLDIERS', 'CDLRISEFALL3METHODS', 'CDLMATHOLD', 
    'CDLSEPARATINGLINES', 'CDLTASUKIGAP',
    
    # Bullish bottom patterns
    'CDLABANDONEDBABY', 'CDLLADDERBOTTOM', 'CDLMATCHINGLOW', 'CDLUNIQUE3RIVER',
    
    # Bullish special patterns
    'CDL3INSIDE', 'CDL3OUTSIDE', 'CDLBELTHOLD', 'CDLBREAKAWAY',
    'CDLKICKING', 'CDLKICKINGBYLENGTH', 'CDLSTICKSANDWICH'
]

print(f"📊 Available candlestick patterns: {len(CANDLESTICK_PATTERNS)}")
for i, pattern in enumerate(CANDLESTICK_PATTERNS, 1):
    print(f"{i:2d}. {pattern}")

In [ ]:
# Test indicator calculations
print("🧪 Testing indicator calculations...")

# Calculate VWAP
df_clean['vwap'] = calculate_vwap(df_clean)
print(f"✅ VWAP calculated - Valid values: {df_clean['vwap'].notna().sum()}/{len(df_clean)}")

# Calculate ATR
df_clean['atr'] = calculate_atr(df_clean, period=14)
print(f"✅ ATR calculated - Valid values: {df_clean['atr'].notna().sum()}/{len(df_clean)}")

# Test a few candlestick patterns
test_patterns = ['CDLHAMMER', 'CDLENGULFING', 'CDLMORNINGSTAR']
for pattern in test_patterns:
    df_clean[pattern.lower()] = calculate_candlestick_pattern(df_clean, pattern)
    valid_signals = (df_clean[pattern.lower()] != 0).sum()
    print(f"✅ {pattern} calculated - Signals found: {valid_signals}")

# Display sample with indicators
print(f"\n📊 Sample data with indicators:")
display(df_clean[['open', 'high', 'low', 'close', 'volume', 'vwap', 'atr']].tail())

## 4. Strategy Configuration and Generation

In [ ]:
def generate_strategy_configs(patterns: List[str]) -> List[Dict]:
    """
    Generate strategy configurations for all VWAP + candlestick combinations
    """
    strategies = []
    
    for pattern in patterns:
        strategy_config = {
            'name': f'VWAP_{pattern}',
            'primary_indicator': 'VWAP',
            'secondary_indicator': pattern,
            'description': f'VWAP combined with {pattern} candlestick pattern',
            'atr_period': 14,
            'sl_multiplier': 1.5,
            'tp_multiplier': 3.0,
            'pattern_function': pattern
        }
        strategies.append(strategy_config)
    
    return strategies

# Generate all strategy configurations
strategy_configs = generate_strategy_configs(CANDLESTICK_PATTERNS)

print(f"🎯 Generated {len(strategy_configs)} strategy configurations:")
print("\n📋 Strategy List:")
for i, config in enumerate(strategy_configs[:10], 1):  # Show first 10
    print(f"{i:2d}. {config['name']} - {config['description']}")

if len(strategy_configs) > 10:
    print(f"    ... and {len(strategy_configs) - 10} more strategies")

print(f"\n📊 Sample strategy configuration:")
display(pd.DataFrame([strategy_configs[0]]).T)

## 5. Time Series Data Splitting

In [ ]:
def create_time_series_splits(df: pd.DataFrame, n_splits: int = 3) -> List[Tuple]:
    """
    Create time series cross-validation splits
    """
    tscv = TimeSeriesSplit(n_splits=n_splits)
    splits = []
    
    print(f"📊 Creating {n_splits} time series splits...")
    
    for i, (train_idx, test_idx) in enumerate(tscv.split(df)):
        train_start = df.index[train_idx[0]]
        train_end = df.index[train_idx[-1]]
        test_start = df.index[test_idx[0]]
        test_end = df.index[test_idx[-1]]
        
        train_data = df.iloc[train_idx]
        test_data = df.iloc[test_idx]
        
        split_info = {
            'split_number': i + 1,
            'train_data': train_data,
            'test_data': test_data,
            'train_period': (train_start, train_end),
            'test_period': (test_start, test_end),
            'train_size': len(train_data),
            'test_size': len(test_data)
        }
        
        splits.append(split_info)
        
        print(f"Split {i+1}:")
        print(f"  📈 Train: {train_start.strftime('%Y-%m-%d')} to {train_end.strftime('%Y-%m-%d')} ({len(train_data)} days)")
        print(f"  🧪 Test:  {test_start.strftime('%Y-%m-%d')} to {test_end.strftime('%Y-%m-%d')} ({len(test_data)} days)")
    
    return splits

# Create time series splits
ts_splits = create_time_series_splits(df_clean, n_splits=3)

# Use the last split for main analysis (largest training set)
main_split = ts_splits[-1]
train_data = main_split['train_data']
test_data = main_split['test_data']

print(f"\n🎯 Using split {main_split['split_number']} for main analysis:")
print(f"📊 Training data: {len(train_data)} days")
print(f"🧪 Testing data: {len(test_data)} days")

## 6. Dynamic Strategy Class Creation

In [ ]:
def create_vwap_strategy_class(pattern_name: str, atr_period: int = 14, 
                              sl_multiplier: float = 1.5, tp_multiplier: float = 3.0):
    """
    Dynamically create a VWAP + Candlestick strategy class
    """
    
    class VWAPCandlestickStrategy(Strategy):
        # Strategy parameters
        atr_period = atr_period
        sl_multiplier = sl_multiplier
        tp_multiplier = tp_multiplier
        pattern_name = pattern_name
        
        def init(self):
            # Calculate VWAP
            typical_price = (self.data.High + self.data.Low + self.data.Close) / 3
            volume_price = typical_price * self.data.Volume
            self.vwap = self.I(lambda: volume_price.cumsum() / self.data.Volume.cumsum(), name='VWAP')
            
            # Calculate ATR
            self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close, 
                             self.atr_period, name='ATR')
            
            # Calculate candlestick pattern
            pattern_func = getattr(ta, self.pattern_name)
            self.pattern = self.I(pattern_func, self.data.Open, self.data.High, 
                                 self.data.Low, self.data.Close, name=self.pattern_name)
            
            # Track entry levels
            self.entry_price = None
            self.stop_loss_level = None
            self.take_profit_level = None
        
        def next(self):
            # Entry conditions: Price above VWAP AND bullish candlestick pattern
            if (not self.position and 
                self.data.Close[-1] > self.vwap[-1] and  # Price above VWAP
                self.pattern[-1] > 0):  # Bullish pattern detected
                
                # Enter long position
                self.buy()
                
                # Set stop loss and take profit levels
                self.entry_price = self.data.Close[-1]
                self.stop_loss_level = self.entry_price - (self.atr[-1] * self.sl_multiplier)
                self.take_profit_level = self.entry_price + (self.atr[-1] * self.tp_multiplier)
            
            # Exit conditions
            if self.position:
                # Stop Loss
                if self.data.Low[-1] <= self.stop_loss_level:
                    self.position.close()
                    self.entry_price = None
                # Take Profit  
                elif self.data.High[-1] >= self.take_profit_level:
                    self.position.close()
                    self.entry_price = None
    
    # Set class name dynamically
    VWAPCandlestickStrategy.__name__ = f"VWAP{pattern_name}Strategy"
    return VWAPCandlestickStrategy

# Test strategy class creation
print("🧪 Testing strategy class creation...")
TestStrategy = create_vwap_strategy_class('CDLHAMMER')
print(f"✅ Created strategy class: {TestStrategy.__name__}")
print(f"📊 Strategy pattern: {TestStrategy.pattern_name}")
print(f"⚙️ Parameters: ATR={TestStrategy.atr_period}, SL={TestStrategy.sl_multiplier}, TP={TestStrategy.tp_multiplier}")

## 7. Batch Backtesting Execution

In [ ]:
def run_single_backtest(strategy_config: Dict, data: pd.DataFrame, 
                       cash: float = 10000, commission: float = 0.002) -> Dict:
    """
    Run a single backtest for a strategy configuration
    """
    try:
        # Prepare data (backtesting library expects uppercase columns)
        df_backtest = data.copy()
        df_backtest.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        
        # Create strategy class
        StrategyClass = create_vwap_strategy_class(
            pattern_name=strategy_config['pattern_function'],
            atr_period=strategy_config['atr_period'],
            sl_multiplier=strategy_config['sl_multiplier'],
            tp_multiplier=strategy_config['tp_multiplier']
        )
        
        # Run backtest
        bt = Backtest(df_backtest, StrategyClass, cash=cash, commission=commission)
        results = bt.run()
        
        # Add strategy info to results
        result_dict = results.to_dict()
        result_dict['strategy_name'] = strategy_config['name']
        result_dict['pattern'] = strategy_config['pattern_function']
        result_dict['backtest_object'] = bt
        result_dict['success'] = True
        result_dict['error'] = None
        
        return result_dict
        
    except Exception as e:
        return {
            'strategy_name': strategy_config['name'],
            'pattern': strategy_config['pattern_function'],
            'success': False,
            'error': str(e),
            'Return [%]': 0,
            '# Trades': 0
        }

def run_batch_backtests(strategy_configs: List[Dict], data: pd.DataFrame, 
                       max_strategies: Optional[int] = None) -> pd.DataFrame:
    """
    Run backtests for multiple strategies
    """
    if max_strategies:
        configs_to_run = strategy_configs[:max_strategies]
        print(f"🎯 Running backtests for first {max_strategies} strategies...")
    else:
        configs_to_run = strategy_configs
        print(f"🎯 Running backtests for all {len(configs_to_run)} strategies...")
    
    results = []
    
    # Run backtests sequentially with progress tracking
    for i, config in enumerate(configs_to_run, 1):
        print(f"⏳ ({i}/{len(configs_to_run)}) Testing {config['name']}...", end=" ")
        
        result = run_single_backtest(config, data)
        results.append(result)
        
        if result['success']:
            trades = result.get('# Trades', 0)
            returns = result.get('Return [%]', 0)
            print(f"✅ {trades} trades, {returns:.2f}% return")
        else:
            print(f"❌ Failed: {result['error']}")
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    # Filter successful backtests
    successful_results = results_df[results_df['success'] == True].copy()
    
    print(f"\n📊 Backtest Summary:")
    print(f"✅ Successful: {len(successful_results)}/{len(results_df)}")
    print(f"❌ Failed: {len(results_df) - len(successful_results)}")
    
    if len(successful_results) > 0:
        print(f"📈 Best performing strategy: {successful_results.loc[successful_results['Return [%]'].idxmax(), 'strategy_name']}")
        print(f"🎯 Best return: {successful_results['Return [%]'].max():.2f}%")
    
    return successful_results

# Run backtests on training data (limit to first 10 strategies for demo)
print("🚀 Starting batch backtesting on training data...")
train_results = run_batch_backtests(strategy_configs, train_data, max_strategies=10)

# Display top performing strategies
if len(train_results) > 0:
    print(f"\n🏆 Top 5 Performing Strategies (Training Data):")
    top_strategies = train_results.nlargest(5, 'Return [%]')[
        ['strategy_name', 'Return [%]', '# Trades', 'Win Rate [%]', 'Sharpe Ratio', 'Max Drawdown [%]']
    ]
    display(top_strategies)

## 8. Out-of-Sample Testing

In [ ]:
def run_out_of_sample_testing(top_results: pd.DataFrame, test_data: pd.DataFrame, 
                             top_n: int = 5) -> pd.DataFrame:
    """
    Run out-of-sample testing on top performing strategies
    """
    # Get top N strategies
    top_strategies = top_results.nlargest(top_n, 'Return [%]')
    
    print(f"🧪 Running out-of-sample testing on top {top_n} strategies...")
    
    oos_results = []
    
    for idx, strategy_row in top_strategies.iterrows():
        strategy_name = strategy_row['strategy_name']
        pattern = strategy_row['pattern']
        
        print(f"⏳ Testing {strategy_name} on out-of-sample data...", end=" ")
        
        # Find original strategy config
        original_config = next(
            (config for config in strategy_configs if config['name'] == strategy_name), 
            None
        )
        
        if original_config:
            # Run backtest on test data
            oos_result = run_single_backtest(original_config, test_data)
            
            if oos_result['success']:
                # Add in-sample performance for comparison
                oos_result['in_sample_return'] = strategy_row['Return [%]']
                oos_result['out_of_sample_return'] = oos_result['Return [%]']
                oos_result['performance_degradation'] = (
                    strategy_row['Return [%]'] - oos_result['Return [%]']
                )
                
                oos_results.append(oos_result)
                print(f"✅ OOS Return: {oos_result['Return [%]']:.2f}% (IS: {strategy_row['Return [%]']:.2f}%)")
            else:
                print(f"❌ Failed: {oos_result['error']}")
        else:
            print("❌ Strategy config not found")
    
    return pd.DataFrame(oos_results)

# Run out-of-sample testing
if len(train_results) > 0:
    oos_results = run_out_of_sample_testing(train_results, test_data, top_n=5)
    
    if len(oos_results) > 0:
        print(f"\n📊 Out-of-Sample Results:")
        oos_comparison = oos_results[[
            'strategy_name', 'in_sample_return', 'out_of_sample_return', 
            'performance_degradation', '# Trades', 'Win Rate [%]'
        ]].copy()
        
        display(oos_comparison)
        
        # Identify robust strategies (minimal performance degradation)
        robust_strategies = oos_comparison[oos_comparison['performance_degradation'] < 10]
        print(f"\n🎯 Robust strategies (< 10% performance degradation): {len(robust_strategies)}")
        if len(robust_strategies) > 0:
            display(robust_strategies)

## 9. Parameter Optimization

In [ ]:
def optimize_strategy_parameters(strategy_config: Dict, data: pd.DataFrame, 
                               optimize_params: Dict) -> Dict:
    """
    Optimize strategy parameters using grid search
    """
    print(f"⚙️ optimizing parameters for {strategy_config['name']}...")
    
    try:
        # Prepare data
        df_backtest = data.copy()
        df_backtest.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        
        # Create base strategy class
        BaseStrategyClass = create_vwap_strategy_class(
            pattern_name=strategy_config['pattern_function']
        )
        
        # Create optimizable strategy class
        class OptimizableStrategy(BaseStrategyClass):
            # Make parameters optimizable
            atr_period = optimize_params.get('atr_period', [14])
            sl_multiplier = optimize_params.get('sl_multiplier', [1.5])
            tp_multiplier = optimize_params.get('tp_multiplier', [3.0])
        
        # Run optimization
        bt = Backtest(df_backtest, OptimizableStrategy, cash=10000, commission=0.002)
        
        optimization_result = bt.optimize(
            atr_period=optimize_params.get('atr_period', [14]),
            sl_multiplier=optimize_params.get('sl_multiplier', [1.5]), 
            tp_multiplier=optimize_params.get('tp_multiplier', [3.0]),
            maximize='Sharpe Ratio',
            max_tries=50
        )
        
        return {
            'strategy_name': strategy_config['name'],
            'optimization_success': True,
            'optimized_params': {
                'atr_period': optimization_result._strategy.atr_period,
                'sl_multiplier': optimization_result._strategy.sl_multiplier,
                'tp_multiplier': optimization_result._strategy.tp_multiplier
            },
            'optimized_results': optimization_result.to_dict(),
            'backtest_object': bt
        }
        
    except Exception as e:
        print(f"❌ Optimization failed: {str(e)}")
        return {
            'strategy_name': strategy_config['name'],
            'optimization_success': False,
            'error': str(e)
        }

# Define parameter ranges for optimization
optimization_params = {
    'atr_period': [10, 14, 20],
    'sl_multiplier': [1.0, 1.5, 2.0],
    'tp_multiplier': [2.0, 3.0, 4.0]
}

# Optimize top 3 strategies
if len(train_results) >= 3:
    top_3_strategies = train_results.nlargest(3, 'Return [%]')
    
    print(f"⚙️ Optimizing parameters for top 3 strategies...")
    optimized_results = []
    
    for idx, strategy_row in top_3_strategies.iterrows():
        strategy_name = strategy_row['strategy_name']
        
        # Find original config
        original_config = next(
            (config for config in strategy_configs if config['name'] == strategy_name),
            None
        )
        
        if original_config:
            opt_result = optimize_strategy_parameters(
                original_config, train_data, optimization_params
            )
            optimized_results.append(opt_result)
    
    # Display optimization results
    successful_optimizations = [r for r in optimized_results if r.get('optimization_success', False)]
    
    if successful_optimizations:
        print(f"\n🎯 Optimization Results:")
        for result in successful_optimizations:
            print(f"\n📊 {result['strategy_name']}:")
            print(f"   Parameters: {result['optimized_params']}")
            opt_return = result['optimized_results'].get('Return [%]', 0)
            opt_sharpe = result['optimized_results'].get('Sharpe Ratio', 0)
            print(f"   Performance: {opt_return:.2f}% return, {opt_sharpe:.2f} Sharpe")
else:
    print("⚠️ Not enough successful strategies for optimization")

## 10. Performance Visualization and Plotting

In [ ]:
def create_comprehensive_plots(results_df: pd.DataFrame, data: pd.DataFrame, 
                              ticker_name: str) -> None:
    """
    Create comprehensive performance visualization plots
    """
    if len(results_df) == 0:
        print("⚠️ No results to plot")
        return
    
    # Set up the plotting environment
    plt.rcParams['figure.figsize'] = (15, 10)
    plt.rcParams['font.size'] = 10
    
    # Create main figure with subplots
    fig = plt.figure(figsize=(20, 15))
    gs = fig.add_gridspec(4, 3, height_ratios=[2, 1, 1, 1], hspace=0.3, wspace=0.3)
    
    # 1. Strategy Performance Comparison
    ax1 = fig.add_subplot(gs[0, :])
    
    # Get top 10 strategies for plotting
    top_strategies = results_df.nlargest(10, 'Return [%]')
    
    # Create bar plot of returns
    bars = ax1.bar(range(len(top_strategies)), top_strategies['Return [%]'], 
                   color='steelblue', alpha=0.7)
    
    # Color bars based on performance
    for i, bar in enumerate(bars):
        returns = top_strategies.iloc[i]['Return [%]']
        if returns > 10:
            bar.set_color('darkgreen')
        elif returns > 0:
            bar.set_color('lightgreen')
        else:
            bar.set_color('lightcoral')
    
    ax1.set_title(f'Top 10 Strategy Performance - {ticker_name}', fontsize=16, fontweight='bold')
    ax1.set_ylabel('Return (%)', fontsize=12)
    ax1.set_xlabel('Strategy', fontsize=12)
    ax1.set_xticks(range(len(top_strategies)))
    ax1.set_xticklabels([name.replace('VWAP_', '') for name in top_strategies['strategy_name']], 
                       rotation=45, ha='right')
    ax1.grid(True, alpha=0.3)
    ax1.axhline(y=0, color='black', linestyle='-', alpha=0.5)
    
    # Add value labels on bars
    for i, bar in enumerate(bars):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{height:.1f}%', ha='center', va='bottom', fontsize=9)
    
    # 2. Risk-Return Scatter Plot
    ax2 = fig.add_subplot(gs[1, 0])
    
    # Filter out invalid values for scatter plot
    valid_data = results_df[
        (results_df['Return [%]'].notna()) & 
        (results_df['Max Drawdown [%]'].notna()) &
        (results_df['Max Drawdown [%]'] != 0)
    ].copy()
    
    if len(valid_data) > 0:
        scatter = ax2.scatter(valid_data['Max Drawdown [%]'].abs(), 
                            valid_data['Return [%]'],
                            c=valid_data['# Trades'], 
                            cmap='viridis', alpha=0.6, s=60)
        
        ax2.set_xlabel('Max Drawdown (%)', fontsize=10)
        ax2.set_ylabel('Return (%)', fontsize=10)
        ax2.set_title('Risk vs Return', fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        
        # Add colorbar
        cbar = plt.colorbar(scatter, ax=ax2)
        cbar.set_label('Number of Trades', fontsize=9)
    
    # 3. Win Rate Distribution
    ax3 = fig.add_subplot(gs[1, 1])
    
    win_rates = results_df['Win Rate [%]'].dropna()
    if len(win_rates) > 0:
        ax3.hist(win_rates, bins=15, alpha=0.7, color='skyblue', edgecolor='black')
        ax3.axvline(win_rates.mean(), color='red', linestyle='--', 
                   label=f'Mean: {win_rates.mean():.1f}%')
        ax3.set_xlabel('Win Rate (%)', fontsize=10)
        ax3.set_ylabel('Frequency', fontsize=10)
        ax3.set_title('Win Rate Distribution', fontsize=12, fontweight='bold')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
    
    # 4. Sharpe Ratio vs Return
    ax4 = fig.add_subplot(gs[1, 2])
    
    sharpe_data = results_df[
        (results_df['Sharpe Ratio'].notna()) & 
        (results_df['Return [%]'].notna())
    ].copy()
    
    if len(sharpe_data) > 0:
        ax4.scatter(sharpe_data['Sharpe Ratio'], sharpe_data['Return [%]'], 
                   alpha=0.6, color='orange', s=60)
        ax4.set_xlabel('Sharpe Ratio', fontsize=10)
        ax4.set_ylabel('Return (%)', fontsize=10)
        ax4.set_title('Sharpe Ratio vs Return', fontsize=12, fontweight='bold')
        ax4.grid(True, alpha=0.3)
        
        # Add trend line
        if len(sharpe_data) > 1:
            z = np.polyfit(sharpe_data['Sharpe Ratio'], sharpe_data['Return [%]'], 1)
            p = np.poly1d(z)
            ax4.plot(sharpe_data['Sharpe Ratio'].sort_values(), 
                    p(sharpe_data['Sharpe Ratio'].sort_values()), 
                    "r--", alpha=0.8, linewidth=1)
    
    # 5. Trade Count Analysis
    ax5 = fig.add_subplot(gs[2, 0])
    
    trade_counts = results_df['# Trades'].dropna()
    if len(trade_counts) > 0:
        ax5.bar(range(len(trade_counts)), sorted(trade_counts, reverse=True), 
               color='lightcoral', alpha=0.7)
        ax5.set_xlabel('Strategy Rank', fontsize=10)
        ax5.set_ylabel('Number of Trades', fontsize=10)
        ax5.set_title('Trade Count by Strategy', fontsize=12, fontweight='bold')
        ax5.grid(True, alpha=0.3)
    
    # 6. Performance Metrics Heatmap
    ax6 = fig.add_subplot(gs[2, 1:])
    
    # Select key metrics for heatmap
    heatmap_metrics = ['Return [%]', 'Sharpe Ratio', 'Win Rate [%]', '# Trades']
    heatmap_data = results_df[heatmap_metrics].head(10)
    
    if len(heatmap_data) > 0:
        # Normalize data for better visualization
        heatmap_normalized = heatmap_data.copy()
        for col in heatmap_metrics:
            if col in heatmap_normalized.columns:
                col_data = heatmap_normalized[col].fillna(0)
                if col_data.std() != 0:
                    heatmap_normalized[col] = (col_data - col_data.min()) / (col_data.max() - col_data.min())
        
        im = ax6.imshow(heatmap_normalized.T, cmap='RdYlGn', aspect='auto')
        
        # Set ticks and labels
        ax6.set_xticks(range(len(heatmap_data)))
        ax6.set_xticklabels([name.replace('VWAP_', '')[:10] + '...' if len(name) > 13 else name.replace('VWAP_', '') 
                           for name in heatmap_data.index], rotation=45, ha='right')
        ax6.set_yticks(range(len(heatmap_metrics)))
        ax6.set_yticklabels(heatmap_metrics)
        ax6.set_title('Performance Metrics Heatmap (Normalized)', fontsize=12, fontweight='bold')
        
        # Add colorbar
        cbar = plt.colorbar(im, ax=ax6)
        cbar.set_label('Normalized Value', fontsize=9)
    
    # 7. Price Chart with VWAP
    ax7 = fig.add_subplot(gs[3, :])
    
    # Plot price and VWAP for context
    recent_data = data.tail(500)  # Last 500 days
    ax7.plot(recent_data.index, recent_data['close'], label=f'{ticker_name} Price', 
            color='blue', linewidth=1, alpha=0.8)
    
    if 'vwap' in recent_data.columns:
        ax7.plot(recent_data.index, recent_data['vwap'], label='VWAP', 
                color='red', linewidth=1.5, alpha=0.8)
    
    ax7.set_title(f'{ticker_name} Price vs VWAP (Last 500 Days)', fontsize=12, fontweight='bold')
    ax7.set_ylabel('Price ($)', fontsize=10)
    ax7.set_xlabel('Date', fontsize=10)
    ax7.legend()
    ax7.grid(True, alpha=0.3)
    
    # Format x-axis dates
    ax7.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    ax7.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    plt.setp(ax7.xaxis.get_majorticklabels(), rotation=45)
    
    plt.suptitle(f'VWAP Strategy Analysis - {ticker_name}', fontsize=18, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.show()
    
    print("✅ Comprehensive plots created successfully!")

# Create plots for training results
if len(train_results) > 0:
    print("📊 Creating comprehensive performance plots...")
    create_comprehensive_plots(train_results, df_clean, ticker_name)
else:
    print("⚠️ No successful backtest results to plot")

In [ ]:
def plot_individual_strategy_performance(strategy_result: Dict, data: pd.DataFrame, 
                                       ticker_name: str) -> None:
    """
    Plot detailed performance for a single strategy
    """
    if not strategy_result.get('success', False):
        print("⚠️ Cannot plot failed strategy")
        return
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # Get the backtest object to access detailed results
    bt = strategy_result.get('backtest_object')
    strategy_name = strategy_result.get('strategy_name', 'Unknown')
    
    # 1. Price chart with entry/exit points (if backtest object available)
    recent_data = data.tail(500)
    ax1.plot(recent_data.index, recent_data['close'], label=f'{ticker_name} Price', 
            color='blue', linewidth=1)
    
    if 'vwap' in recent_data.columns:
        ax1.plot(recent_data.index, recent_data['vwap'], label='VWAP', 
                color='red', linewidth=1.5, alpha=0.7)
    
    ax1.set_title(f'{strategy_name} - Price & VWAP', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Price ($)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Performance metrics bar chart
    metrics = {
        'Return (%)': strategy_result.get('Return [%]', 0),
        'Sharpe Ratio': strategy_result.get('Sharpe Ratio', 0),
        'Win Rate (%)': strategy_result.get('Win Rate [%]', 0),
        'Max DD (%)': -abs(strategy_result.get('Max Drawdown [%]', 0))
    }
    
    colors = ['green' if v > 0 else 'red' for v in metrics.values()]
    bars = ax2.bar(metrics.keys(), metrics.values(), color=colors, alpha=0.7)
    ax2.set_title('Key Performance Metrics', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Value')
    ax2.grid(True, alpha=0.3)
    
    # Add value labels
    for bar, value in zip(bars, metrics.values()):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + (0.5 if height > 0 else -0.5),
                f'{value:.1f}', ha='center', va='bottom' if height > 0 else 'top')
    
    # 3. Trade statistics
    trade_stats = {
        'Total Trades': strategy_result.get('# Trades', 0),
        'Winning Trades': int(strategy_result.get('# Trades', 0) * strategy_result.get('Win Rate [%]', 0) / 100),
        'Avg Trade (%)': strategy_result.get('Avg Trade [%]', 0),
        'Best Trade (%)': strategy_result.get('Best Trade [%]', 0),
        'Worst Trade (%)': strategy_result.get('Worst Trade [%]', 0)
    }
    
    ax3.barh(list(trade_stats.keys()), list(trade_stats.values()), color='skyblue', alpha=0.7)
    ax3.set_title('Trade Statistics', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Value')
    ax3.grid(True, alpha=0.3)
    
    # 4. Risk metrics
    risk_metrics = {
        'Volatility (%)': strategy_result.get('Volatility (ann.) [%]', 0),
        'Max Drawdown (%)': abs(strategy_result.get('Max Drawdown [%]', 0)),
        'Calmar Ratio': strategy_result.get('Calmar Ratio', 0) if strategy_result.get('Calmar Ratio') not in ['N/A', None] else 0,
        'Sortino Ratio': strategy_result.get('Sortino Ratio', 0) if strategy_result.get('Sortino Ratio') not in ['N/A', None] else 0
    }
    
    ax4.pie([max(0.1, abs(v)) for v in risk_metrics.values()], 
           labels=risk_metrics.keys(), autopct='%1.1f%%', startangle=90)
    ax4.set_title('Risk Profile', fontsize=14, fontweight='bold')
    
    plt.suptitle(f'Detailed Analysis: {strategy_name}', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Plot detailed analysis for best strategy
if len(train_results) > 0:
    best_strategy = train_results.loc[train_results['Return [%]'].idxmax()].to_dict()
    print(f"📊 Creating detailed analysis for best strategy: {best_strategy['strategy_name']}")
    plot_individual_strategy_performance(best_strategy, df_clean, ticker_name)

## 11. Monte Carlo Simulation and Robustness Testing

In [ ]:
def run_monte_carlo_simulation(strategy_config: Dict, original_data: pd.DataFrame, 
                              n_simulations: int = 100, random_seed: int = 42) -> Dict:
    """
    Run Monte Carlo simulation for strategy robustness testing
    """
    np.random.seed(random_seed)
    random.seed(random_seed)
    
    print(f"🎲 Running Monte Carlo simulation for {strategy_config['name']} ({n_simulations} runs)...")
    
    results = []
    
    # Generate random OHLC data based on original data statistics
    ohlc_generator = random_ohlc_data(original_data[['open', 'high', 'low', 'close', 'volume']])
    
    for i in range(n_simulations):
        try:
            # Generate random data
            random_data = next(ohlc_generator)
            random_data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
            
            # Run backtest on random data
            result = run_single_backtest(strategy_config, random_data)
            
            if result['success']:
                results.append({
                    'simulation': i + 1,
                    'return': result.get('Return [%]', 0),
                    'sharpe': result.get('Sharpe Ratio', 0),
                    'max_dd': result.get('Max Drawdown [%]', 0),
                    'trades': result.get('# Trades', 0),
                    'win_rate': result.get('Win Rate [%]', 0)
                })
            
            if (i + 1) % 20 == 0:
                print(f"  ⏳ Completed {i + 1}/{n_simulations} simulations...")
                
        except Exception as e:
            continue
    
    if len(results) == 0:
        return {'success': False, 'error': 'No successful simulations'}
    
    # Calculate statistics
    returns = [r['return'] for r in results]
    sharpes = [r['sharpe'] for r in results if not pd.isna(r['sharpe'])]
    
    mc_stats = {
        'success': True,
        'n_successful_sims': len(results),
        'return_stats': {
            'mean': np.mean(returns),
            'std': np.std(returns),
            'min': np.min(returns),
            'max': np.max(returns),
            'percentile_5': np.percentile(returns, 5),
            'percentile_95': np.percentile(returns, 95)
        },
        'sharpe_stats': {
            'mean': np.mean(sharpes) if sharpes else 0,
            'std': np.std(sharpes) if sharpes else 0,
        },
        'positive_return_probability': len([r for r in returns if r > 0]) / len(returns),
        'raw_results': results
    }
    
    print(f"✅ Monte Carlo completed: {len(results)} successful simulations")
    print(f"📊 Mean return: {mc_stats['return_stats']['mean']:.2f}% ± {mc_stats['return_stats']['std']:.2f}%")
    print(f"🎯 Positive return probability: {mc_stats['positive_return_probability']:.2%}")
    
    return mc_stats

def plot_monte_carlo_results(mc_results: Dict, strategy_name: str) -> None:
    """
    Plot Monte Carlo simulation results
    """
    if not mc_results.get('success', False):
        print("⚠️ No Monte Carlo results to plot")
        return
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    results = mc_results['raw_results']
    returns = [r['return'] for r in results]
    sharpes = [r['sharpe'] for r in results if not pd.isna(r['sharpe'])]
    
    # 1. Return distribution
    ax1.hist(returns, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    ax1.axvline(np.mean(returns), color='red', linestyle='--', 
               label=f'Mean: {np.mean(returns):.2f}%')
    ax1.axvline(0, color='black', linestyle='-', alpha=0.5)
    ax1.set_xlabel('Return (%)')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Monte Carlo Return Distribution')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Cumulative return probability
    sorted_returns = np.sort(returns)
    cumprob = np.arange(1, len(sorted_returns) + 1) / len(sorted_returns)
    
    ax2.plot(sorted_returns, cumprob, color='green', linewidth=2)
    ax2.axvline(0, color='red', linestyle='--', alpha=0.7, label='Break-even')
    ax2.set_xlabel('Return (%)')
    ax2.set_ylabel('Cumulative Probability')
    ax2.set_title('Cumulative Return Probability')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Sharpe ratio distribution (if available)
    if sharpes:
        ax3.hist(sharpes, bins=20, alpha=0.7, color='orange', edgecolor='black')
        ax3.axvline(np.mean(sharpes), color='red', linestyle='--',
                   label=f'Mean: {np.mean(sharpes):.2f}')
        ax3.set_xlabel('Sharpe Ratio')
        ax3.set_ylabel('Frequency')
        ax3.set_title('Sharpe Ratio Distribution')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
    else:
        ax3.text(0.5, 0.5, 'No Sharpe Ratio Data', ha='center', va='center', 
                transform=ax3.transAxes, fontsize=14)
        ax3.set_title('Sharpe Ratio Distribution')
    
    # 4. Risk-return scatter
    max_dds = [abs(r['max_dd']) for r in results]
    ax4.scatter(max_dds, returns, alpha=0.6, color='purple')
    ax4.set_xlabel('Max Drawdown (%)')
    ax4.set_ylabel('Return (%)')
    ax4.set_title('Risk vs Return')
    ax4.grid(True, alpha=0.3)
    
    plt.suptitle(f'Monte Carlo Analysis: {strategy_name}', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Run Monte Carlo simulation on best strategy
if len(train_results) > 0:
    best_strategy_name = train_results.loc[train_results['Return [%]'].idxmax(), 'strategy_name']
    best_config = next(
        (config for config in strategy_configs if config['name'] == best_strategy_name),
        None
    )
    
    if best_config:
        # Prepare data in correct format for Monte Carlo
        mc_data = df_clean[['open', 'high', 'low', 'close', 'volume']].copy()
        mc_data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        
        print(f"🎲 Starting Monte Carlo simulation for best strategy: {best_strategy_name}")
        mc_results = run_monte_carlo_simulation(best_config, mc_data, n_simulations=50)
        
        if mc_results.get('success', False):
            plot_monte_carlo_results(mc_results, best_strategy_name)
        else:
            print(f"❌ Monte Carlo simulation failed: {mc_results.get('error', 'Unknown error')}")
else:
    print("⚠️ No strategies available for Monte Carlo simulation")

## 12. Multi-Timeframe Analysis